In [1]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [2]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)

In [3]:
alldats = get_dats(range(1573, 1637))

In [11]:
# Hack since I am lazy and there are many CSQ traces mixed in
dats = []
for dat in alldats:
    if dat.Logs.sweeprate > 500:
        dats.append(dat)

In [12]:
Θ = dats[0].Transition.avg_fit.best_values.theta 
fit = dats[0].Transition.get_fit(which='avg')
params = fit.params
params.add('g', value=0, vary=True, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=False) # Based on some old DC bias scans
print(new_pars)

Parameters([('amp', <Parameter 'amp', value=1.0717699436262165 +/- 0.000357, bounds=[0:inf]>), ('const', <Parameter 'const', value=5.638922819555451 +/- 7.5e-05, bounds=[-inf:inf]>), ('lin', <Parameter 'lin', value=0.0016655546243076191 +/- 1.51e-06, bounds=[0:inf]>), ('mid', <Parameter 'mid', value=0.00022379153539901927 +/- 0.00692, bounds=[-inf:inf]>), ('theta', <Parameter 'theta', value=3.958513677885228 (fixed), bounds=[0.01:inf]>), ('g', <Parameter 'g', value=0, bounds=[-50:1000]>)])


In [26]:
amp_digamma_ = [narrow_fit(
    dat,
    200,
    which='avg', 
    initial_params=params, 
    fit_func=i_sense_digamma, 
    check_exists=True).best_values.amp
for dat in progressbar(dats)]

100% (30 of 30) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [27]:
g_digamma_ = [narrow_fit(
    dat,
    200,
    which='avg', 
    initial_params=params, 
    fit_func=i_sense_digamma, 
    check_exists=True).best_values.g
for dat in progressbar(dats)]

100% (30 of 30) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [28]:
escs = [dat.Logs.fds["ESC"] for dat in dats]

In [31]:
fig = go.Figure()
datnums = [dat.datnum for dat in dats]
ind_remove = [np.argmin(np.abs(np.subtract(datnums, num))) for num in [1592, 1598, 1632, 1634]]
temp_escs = []
temp_amps = []
for i, dat in enumerate(dats):
    if i not in ind_remove:
        temp_escs.append(escs[i])
        temp_amps.append(amp_digamma_[i])
fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_, text=[dat.datnum for dat in dats], name="Data"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Amplitude /nA',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
res = np.polyfit(temp_escs, temp_amps, 2)
fit = np.polyval(res, escs)
print(res)
fig.add_trace(go.Scatter(mode='lines', x=escs, y=fit, name="Fit"))
fig.show(renderer="browser")

[-2.85721853e-05 -1.80534391e-02 -1.75449395e+00]


In [32]:
fig.write_html("Exports/Amplitude_03_06_narrow_fitting.html")

In [47]:
fig = go.Figure()
for dat in dats:
    x = dat.Transition.avg_x
    y = dat.Transition.avg_data
    yfit = narrow_fit(
            dat,
            200,
            which='avg', 
            initial_params=params, 
            fit_func=i_sense_digamma, 
            check_exists=True).eval_fit(x)
    fig.add_trace(go.Scatter(mode='markers', x=x[::100], y=y[::100], name=f'{dat.datnum}d'))
    fig.add_trace(go.Scatter(mode='lines', x=x[::25], y=yfit[::25], name=f'{dat.datnum}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [48]:
fig.write_html("Exports/AllFits_03_06_narrow_fitting.html")

In [46]:
def narrow_fit(dat, width, **kwargs):
    ''' 
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = dat.Transition.avg_x
    y = dat.Transition.avg_data
    
    start_ind = np.argmin(np.abs(np.add(x, width)))
    end_ind = np.argmin(np.abs(np.subtract(x, width)))
    
    return dat.Transition.get_fit(
            x = x[start_ind:end_ind],
            data = y[start_ind:end_ind],
            **kwargs)

In [57]:
fig = go.Figure()
for dat in [dats[28], dats[29]]:
    x = dat.Transition.avg_x
    y = dat.Transition.avg_data
    yfit = narrow_fit(
        dat,
        500,
        which='avg', 
        initial_params=params, 
        fit_func=i_sense_digamma, 
        check_exists=False).eval_fit(x)
    fig.add_trace(go.Scatter(mode='markers', x=x[::100], y=y[::100], name=f'{dat.datnum}d'))
    fig.add_trace(go.Scatter(mode='lines', x=x[::25], y=yfit[::25], name=f'{dat.datnum}f', marker_color='grey'))
    print(narrow_fit(
        dat,
        500,
        which='avg', 
        initial_params=params, 
        fit_func=i_sense_digamma, 
        check_exists=False).best_values.amp)
fig.show(renderer="browser")

0.5361376289776723
0.47050325908829005


In [55]:
fig.write_html("Exports/VeryGammaBroadened_03_06_narrrow.html")

In [41]:
for i, dat in enumerate(dats):
    print(f'{i}: {dat.datnum}')

0: 1574
1: 1576
2: 1578
3: 1580
4: 1582
5: 1584
6: 1586
7: 1588
8: 1590
9: 1592
10: 1594
11: 1596
12: 1598
13: 1600
14: 1604
15: 1606
16: 1608
17: 1610
18: 1612
19: 1614
20: 1616
21: 1618
22: 1620
23: 1622
24: 1624
25: 1626
26: 1628
27: 1630
28: 1632
29: 1634
